In [1]:
import sys
sys.path.insert(1, r'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks')
from bb_setup import setup_1_old,main_features

In [2]:
import pandas as pd
import talib as ta
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")

In [443]:
time_period = 20
var_bb = 2.5
rate_tp = 2.5
rate_stop = 1
pontos = 0.0010
spread = 0.000005

timeframe = 1
ativo = f'EURUSD{timeframe}'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(fr'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks\Dados\{ativo}.csv', names = colnames, sep='\t')
#df = df[df['Data'] > '2022-01-01']

df = main_features(df)

df_acao = setup_1_old(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)


In [444]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 1 --pontos: 0.001 --- rate_tp: 2.5 -- var_bb: 2.5
Pontos_liquido: 0.27987 -- Total_operacoes: 4026
Agrupado por Hora
   Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0      00            161     0.298137        0.006195
1      01            159     0.276730       -0.005795
2      02            146     0.397260        0.056270
3      03            189     0.301587        0.009555
4      04            194     0.293814        0.004530
5      05            190     0.363158        0.050550
6      06            167     0.281437       -0.003335
7      07            156     0.301282        0.007720
8      08            154     0.376623        0.048230
9      09            158     0.291139        0.002210
10     10            158     0.265823       -0.011790
11     11            172     0.343023        0.033640
12     12            179     0.273743       -0.008395
13     13            162     0.302469        0.008690
14     14            142     0.260563       -0.013210
15     1

#### Modelo

In [412]:
df_acao = df_acao[df_acao['Hora_h'].isin(['01','04','10','11','12','14','15','17','21','23'])]

In [445]:
df_modelo = df_acao[['Data','acao','Vol','macdhist','dist_bb','dist_bb_up','dist_bb_low','dist_media','dist_media_abs',
'dist_price_media_21','dist_price_media_5','dist_RSI_70','dist_RSI_30','tamanho_corpo','tamanho_full',
'media_vol_5','media_vol_3','media_vol_10','tamanho_corpo_5','tamanho_corpo_direcao','candle_direcao','candle_direcao_5','resultado_binario']]

In [413]:
df_modelo = df_acao[['Data','acao','media_vol_10','macdhist','dist_bb_low','tamanho_full','dist_price_media_5',
'media_vol_5','Vol','resultado_binario']]

In [311]:
df_modelo = df_modelo[df_modelo['acao'] == 'call']

In [446]:
inicio = '2021-01-01'
termino = '2022-09-01'
df_modelo = df_modelo.dropna()

df_train = df_modelo[(df_modelo['Data'] > inicio) & (df_modelo['Data'] < termino)]
df_train = df_train.set_index(['Data','acao'])
df_train=(df_train-df_train.min())/(df_train.max()-df_train.min())

df_1 = df_train[df_train['resultado_binario'] == 1]
df_0 = df_train[df_train['resultado_binario'] == 0].iloc[:df_1.shape[0]-100]

df_train = pd.concat([df_1,df_0])
df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test = df_modelo[(df_modelo['Data'] > termino)]
df_test = df_test.set_index(['Data','acao'])
df_test=(df_test-df_test.min())/(df_test.max()-df_test.min())

y_train = df_train['resultado_binario']
x_train = df_train.drop(columns=['resultado_binario'])

y_test = df_test['resultado_binario']
x_test = df_test.drop(columns=['resultado_binario'])

In [447]:
df_train['resultado_binario'].sum()/df_train.shape[0]

0.5332005312084993

In [448]:
df_test['resultado_binario'].sum()/df_test.shape[0]

0.2913862718707941

In [436]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb

In [453]:
#clf = ExtraTreesClassifier()
#clf = XGBClassifier()
clf = lgb.LGBMClassifier()
clf.fit(x_train, y_train)

# predict the results
y_pred=clf.predict(x_test)
y_proba = clf.predict_proba(x_test)[:,1]
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

LightGBM Model accuracy score: 0.4105
Training set score: 0.9940
Test set score: 0.4105


In [454]:
df_thr = x_test.copy()
df_thr = df_thr.reset_index()
df_thr['pred'] = y_pred
df_thr['proba'] = y_proba
df_thr['resultado_binario'] = y_test.values

lista_full = []
for thr in [0.5,0.6,0.7,0.8,0.9]:

    df_temp = df_thr[df_thr['proba'] > thr]

    qtd_operacoes = df_temp.shape[0]

    acc = df_temp[df_temp['resultado_binario'] ==1].shape[0]/qtd_operacoes



    lista_full.append([thr,qtd_operacoes,acc])

df_a = pd.DataFrame(lista_full)
df_a.columns = ['thr','qtd_operacoes','acc']
df_a

,thr,qtd_operacoes,acc
0,0.5,1117,0.301701
1,0.6,965,0.301554
2,0.7,759,0.310935
3,0.8,541,0.299445
4,0.9,293,0.296928


In [439]:
pd.Series(clf.feature_importances_, index=x_test.columns).nlargest(20)

tamanho_corpo_direcao    0.076147
dist_bb                  0.066720
media_vol_3              0.066500
dist_price_media_21      0.064100
media_vol_5              0.061759
Vol                      0.061430
media_vol_10             0.059513
tamanho_full             0.055814
dist_bb_up               0.054266
tamanho_corpo_5          0.053588
dist_media_abs           0.051703
macdhist                 0.051556
tamanho_corpo            0.051297
dist_bb_low              0.050613
dist_RSI_70              0.046018
candle_direcao_5         0.044911
dist_media               0.042861
dist_price_media_5       0.041206
candle_direcao           0.000000
dist_RSI_30              0.000000
dtype: float32

In [455]:
df_acao_nova = df_thr[df_thr['proba'] > 0.7]

In [456]:
qtd_operacoes = df_acao_nova.shape[0]
taxa_acerto = df_acao_nova['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao_nova['Hora'] = df_acao_nova['Data'].apply(lambda x: x[11:])
df_acao_nova['Hora_h'] = df_acao_nova['Hora'].apply(lambda x: x[:2])
df_acao_nova['Mes'] = df_acao_nova['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao_nova[['Mes','resultado_binario']]
df_new_hora = df_acao_nova[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 1 --pontos: 0.001 --- rate_tp: 2.5 -- var_bb: 2.5
Pontos_liquido: 0.063205 -- Total_operacoes: 759
Agrupado por Hora
   Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0      00             29     0.344828        0.005855
1      01             28     0.178571       -0.010640
2      02             32     0.500000        0.023840
3      03             23     0.478261        0.015385
4      04             39     0.307692        0.002805
5      05             35     0.314286        0.003325
6      06             52     0.211538       -0.013760
7      07             37     0.243243       -0.005685
8      08             39     0.384615        0.013305
9      09             29     0.379310        0.009355
10     10             42     0.309524        0.003290
11     11             47     0.255319       -0.005235
12     12             45     0.355556        0.010775
13     13             49     0.285714       -0.000245
14     14             36     0.138889       -0.018680
15     1

In [457]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 1 --pontos: 0.001 --- rate_tp: 2.5 -- var_bb: 2.5
Pontos_liquido: 0.27987 -- Total_operacoes: 4026
Agrupado por Hora
   Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0      00            161     0.298137        0.006195
1      01            159     0.276730       -0.005795
2      02            146     0.397260        0.056270
3      03            189     0.301587        0.009555
4      04            194     0.293814        0.004530
5      05            190     0.363158        0.050550
6      06            167     0.281437       -0.003335
7      07            156     0.301282        0.007720
8      08            154     0.376623        0.048230
9      09            158     0.291139        0.002210
10     10            158     0.265823       -0.011790
11     11            172     0.343023        0.033640
12     12            179     0.273743       -0.008395
13     13            162     0.302469        0.008690
14     14            142     0.260563       -0.013210
15     1